In [3]:
# import libraries
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

# read in data, taking 10% sample
p = .1
required_cols = ['JobID','Partition', 'Account', 'Group', 'GID',
       'User', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed',
       'ExitCode', 'State', 'NNodes', 'NCPUS', 'ReqCPUS', 'ReqMem', 'ReqGRES',
       'ReqTRES', 'Timelimit', 'NodeList', 'JobName']
accre = pd.read_csv('/data-question-2-the_machine/data/accre_data.txt', header=0, sep='|', usecols=required_cols, skiprows=lambda i: i>0 and random.random() > p)

FileNotFoundError: File b'/data-question-2-the_machine/data/accre_data.txt' does not exist

## DISCOVERY

In [ ]:
accre.info()

In [ ]:
# look at the shape of the dataset
accre.shape

## CLEANUP

In [ ]:
# convert partition & account to category
accre.Partition = accre.Partition.astype('category')
accre.Account = accre.Account.astype('category')
accre.dtypes

In [ ]:
# convert Submit and Start to datetime
accre['Submit'] = pd.to_datetime(accre['Submit'], format='%Y/%m/%d')
accre['Start'] = pd.to_datetime(accre['Start'], format='%Y/%m/%d')

In [ ]:
# create Queue_Time column = Start - Submit
accre['Queue_Time'] = accre['Start'] - accre['Submit']
accre.Queue_Time.head()

In [ ]:
# check data types
accre.info()

In [ ]:
# head data exploration for each column
for column in list(accre.columns):
    print(accre[column].value_counts(normalize=False,ascending=False).head(10))
    print('')
    print('')
    

## BEGIN WORK ON QUESTION

In [1]:
#get count of status
state_messages = accre.State.value_counts().to_frame().reset_index()
state_messages.columns = ["status","count"]
state_messages.head(20)


NameError: name 'accre' is not defined

In [ ]:
#group cancelled statuses
import re
condition = re.compile("CANCELLED by\s\d*")
state_messages['status'] = state_messages.status.replace(to_replace=condition,value="CANCELLED")
state_messages.groupby("status")["count"].sum().to_frame().reset_index()

In [ ]:
# get jobs run per node (count of JobID grouped by NodeList)
jobs_per_node = accre.groupby('NodeList').JobID.count().to_frame().sort_values(by="JobID",ascending=False).iloc[1:,:].head()
jobs_per_node

In [ ]:
#jobs failed per node (count of JobID grouped by NodeList from )
state_failed = ['NODE_FAIL']
state_bad = accre.loc[accre['State'].isin(state_failed)]
fails_per_node = state_bad.groupby('NodeList').JobID.count().to_frame().sort_values("JobID",ascending=False)
fails_per_node.head()

In [ ]:
#merge jobs run and jobs failed
node_fails_table = pd.merge(jobs_per_node,fails_per_node,on='NodeList',how='left')
node_fails_table = node_fails_table.rename({'JobID_x':'JobRuns','JobID_y':'JobFails'},axis=1)
node_fails_table.head()

In [ ]:
#drop rows missing data
##should first check how many rows that will be
node_fails_table = node_fails_table.dropna()

#create Fail% column & sort
node_fails_table['Fail%'] = node_fails_table.JobFails/node_fails_table.JobRuns
node_fails_table.sort_values(by='JobRuns', ascending=False)

In [ ]:
#drop rows with no jobs failed or <100 jobs run
condition1 = node_fails_table.JobFails != 0
condition2 = node_fails_table.JobRuns > 100

node_fails_table = node_fails_table[condition1&condition2]

In [ ]:
#show sorted table
node_fails_table = node_fails_table.sort_values(by='Fail%',ascending=False)
node_fails_table

In [ ]:
#barplot of top 5 
node_fails_table[["Fail%"]].iloc[0:5,:].plot(kind="bar")